In [2]:
!pip install google-adk

In [3]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print('Gemini API key setup complete')
except Exception as e:
    print(
        f"Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

Gemini API key setup complete


In [4]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

- AI Agent — a program that can think, reason, and use tools (like web search or APIs)
- Gemini, Google’s LLM provides the AI brain to agent to understand prompts and generate text.
- InMemoryRunner executes your agent and manages the conversation flow and keeps all agent interactions in memory (not saved to disk or cloud).
- google_search is built-in tool for your agent to perform real-time Google searches.
- types provides data types and schemas for Googles GenAI API (like prompt, response, model settings) used to define structured inputs/outputs for your LLM calls.

In [5]:
# When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability
# Retry options automatically handle these failures by retrying the request with exponential backoff

retry_config=types.HttpRetryOptions(
    attempts=5,  # maximum retry attempts
    exp_base=7,  # delay multiplier
    initial_delay=1, # initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # retry on these HTTP errors
)

What is an AI Agent?¶

we have used an LLM like Gemini before, where you give it a prompt and it gives you a text response.

Prompt -> LLM -> Text

An AI Agent takes this one step further. An agent can think, take actions, and observe the results of those actions to give you a better answer.

Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer

Define your agent <br>
lets build our agent. We will configure an Agent by setting its key properties, which tell it what to do and how to operate.

To learn more, check out the documentation related to [agents in ADK.](https://google.github.io/adk-docs/agents/)

These are the main properties we will set:

name and description: A simple name and description to identify our agent. <br>
model: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite". <br>
instruction: The agent's guiding prompt. This tells the agent what its goal is and how to behave. <br>
tools: A list of tools that the agent can use. To start, we'll give it the google_search tool, which lets it find up-to-date information online.

In [6]:
root_agent = Agent(name = 'helpful_assistant', model = Gemini(model='gemini-2.5-flash-lite',retry_options=retry_config),
    description='A simple agent that can answer general questions',
    instruction='You are a helpful assistant. Use Google Search for current info or if unsure',
    tools=[google_search],
)

print('Root Agent defined succesfully')

Root Agent defined succesfully


In [7]:
# Runner is central component within ADK that acts as the orchestrator. 
# It manages the conversation, sends our messages to the agent, and handles its responses.

# from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=root_agent)

print('Runner created')

Runner created


In [8]:
response = await runner.run_debug(
    'What is Agent Development Kit from Google? What languages is the SDK available in ? How does agents started in the field of AI'
)


 ### Created new session: debug_session_id

User > What is Agent Development Kit from Google? What languages is the SDK available in ? How does agents started in the field of AI
helpful_assistant > The Google Agent Development Kit (ADK) is a flexible and modular framework designed to simplify the creation, deployment, and orchestration of AI agents and multi-agent systems. It applies software development principles to AI agent creation, allowing developers to build production-ready agentic applications with greater flexibility and control. ADK is model-agnostic and compatible with various frameworks, although it is optimized for the Google ecosystem, particularly with Gemini models and Vertex AI.

The SDK is currently available in the following languages:
*   **Python**
*   **Java**
*   **Go**

The journey of agents in the field of AI began with simple, rule-based systems in the mid-20th century. These early agents, like ELIZA (a chatbot developed in 1966), were designed to mimic huma

In [9]:
response = await runner.run_debug('how to integrate SAP BODS, S4/HANA and Generative AI')


 ### Continue session: debug_session_id

User > how to integrate SAP BODS, S4/HANA and Generative AI
helpful_assistant > While integrating SAP BODS, S/4HANA, and Generative AI (GenAI) is a complex process, it's achievable through SAP's Business Technology Platform (BTP) and specific AI services. Here's a breakdown of how these components can work together:

**1. SAP Business Technology Platform (BTP) as the Foundation:**
SAP BTP serves as the central hub for integrating GenAI capabilities with SAP systems. It provides the necessary infrastructure, tools, and services to develop, deploy, and manage AI-powered applications.

*   **Generative AI Hub:** This component within BTP offers access to various large language models (LLMs) from different providers (e.g., Google Gemini, Microsoft Azure OpenAI, AWS) and SAP's own foundation models. It facilitates prompt engineering and allows for the selection of the most suitable model for specific use cases.
*   **SAP AI Core and SAP AI Launchpad

How does it work?

The agent performed a Google Search to get the latest information about ADK, and it knew to use this tool because:

- The agent inspects and is aware of which tools it has available to use.
- The agent's instructions specify the use of the search tool to get current information or if it is unsure of an answer.
- The best way to see the full, detailed trace of the agent's thoughts and actions is in the ADK web UI, which we'll set up later in this notebook.


-----------------------------------------

# ADK includes built-in web interface for interactively chatting with, testing, debugging your agents

To use the ADK web UI, you will need to create an agent with Python files using the adk create command.

Run the command below to generate a sample-agent folder that contains all the necessary files, <br>
including agent.py for your code, an .env file with your API key pre-configured, and an __init__.py file:

In [10]:
!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY


Agent created in /kaggle/working/sample-agent:
- .env
- __init__.py
- agent.py



In [12]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong> IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("Helper functions defined.")

Helper functions defined.


In [13]:
url_prefix = get_adk_proxy_url()

In [14]:
!adk web --url_prefix {url_prefix}

/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [154]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http: